# Wärmeleitung
Wärmeleitung bzw. Konduktion wird durch einen Temperaturgradienten angetrieben. Damit handelt es sich um einen diffusive Bewegung. Dieses Notebook löst die Diffusionsgleichung um die 1D Wärmeleitung als Funktion von Ort (x) und Zeit (t) zu beschreiben.

Die Grundwasserströmung verhält sich ähnlich und kann mit den gleichen Gesetzen für die 1D-Ausbreitung beschrieben werden.

# Gleichung für die 1D Wärmeleitung (in porösen Medien / Wasser-Festoff)

heat capacity = $c$

density = $\rho$

thermal conductivity = $\lambda$

Temperature = $T$

$D_h=\frac{n_e \lambda_w + (1-n_e)\lambda_s}{n_e c_w \rho_w}$ with $D_h$ = thermal Diffusivity

1-D Conduction without heat storage

$T(x,t)=T_0 erfc (\frac{x}{\sqrt{4 D_h t}})$

In [10]:
# Initialize librarys
from scipy.special import erfc, erf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
from ipywidgets import *

In [16]:
# Define functions
def D_h(lam_w, lam_s, rho_w, c_w, ne):
    nrhowcw = ne *rho_w*c_w
    D_h = (ne*lam_w+(1-ne)*lam_s)/nrhowcw
    return D_h

def RET(c_s, c_w, rho_w, rho_s, ne):
    KD = c_s/(c_w*rho_w)
    RET = 1 + (1-ne)/ne*rho_s*KD
    return RET
    

def Flow1D(t_max,x_max,Tini,dT,lam_s,c_s,rho_s,lam_w,c_w,rho_w,ne,x1,t2,):
    t1 = np.arange(1, t_max, 1)
    x2 = np.arange(1, x_max,1)
    t_maxd = t_max/365
    T1=Tini+dT*erfc(x1/(4*D_h(lam_w, lam_s, rho_w, c_w, ne)*(t1*86400))**0.5)
    T2=Tini+dT*erfc(x2/(4*D_h(lam_w, lam_s, rho_w, c_w, ne)*(t2*86400))**0.5)
    T3=Tini+dT*erfc(x1/(4*D_h(lam_w, lam_s, rho_w, c_w, ne)*(t1*86400)/RET(c_s, c_w, rho_w, rho_s, ne))**0.5)
    T4=Tini+dT*erfc(x2/(4*D_h(lam_w, lam_s, rho_w, c_w, ne)*(t2*86400)/RET(c_s, c_w, rho_w, rho_s, ne))**0.5)
    
    #PLOT
    fig = plt.figure(figsize=(18,12))
    gs = fig.add_gridspec(2,2)
    ax1 = fig.add_subplot(gs[0,0])
    ax2 = fig.add_subplot(gs[0,1])
    
    ax1.plot(t1,T1, color='r')
    ax1.set(xlabel='Zeit (d)', ylabel='Temperatur (m)',title='Temperatur als Funktion des Ortes')
    ax1.grid()
    ax1.set_ylim(Tini-2,dT+Tini+2)
    ax1.set_xlim(0,t_max)
    ax1.text(t_max*0.95, (dT+Tini+2)*0.9, 'x: {} m '.format(x1), horizontalalignment='right', bbox=dict(boxstyle="square", facecolor='grey'))
   
    ax2.plot(x2,T2,color='g')
    ax2.set(xlabel='Ort (x)', ylabel='Temperatur (m)',title='Temperatur als Funktion der Zeit')
    ax2.grid()
    ax2.set_ylim(Tini-2,dT+Tini+2)
    ax2.set_xlim(0,x_max)
    ax2.text(x_max*0.95, (dT+Tini+2)*0.9, 't: {} d '.format(t2), horizontalalignment='right', bbox=dict(boxstyle="square", facecolor='grey'))
   
    plt.show()
    
    print ('D_h =', D_h(lam_w, lam_s, rho_w, c_w, ne))
    print ("R =",  RET(c_s, c_w, rho_w, rho_s, ne))


In [18]:
t_max = 365
x_max = 100

interact(Flow1D,
         t_max=fixed(t_max),
         x_max=fixed(x_max),
         Tini=widgets.FloatSlider(value=   8,     min=4,  max=15,    step=0.1, description = "Temp. ini (C)", readout=True,readout_format='.2f'),
         dT  =widgets.FloatSlider(value=   5,   min=0.1,  max=15,    step=0.1, description = "Delta Temp (C)", readout=True,readout_format='.2f'),
         lam_s =widgets.FloatSlider(value=2.000,  min=0.3,    max=3, step=0.001, description = "Lambda S", readout=True,readout_format='.3f'),
         c_s   =widgets.IntSlider  (value= 840,min=0.200, max=5000,     step=1, description = "Cs"),
         rho_s=widgets.FloatSlider(value= 2650,  min=1800,    max=3500, step=0.1, description = "Rho_s",readout=True,readout_format='.3f'),
         lam_w =widgets.FloatSlider(value=0.598,  min=0.3,    max=3, step=0.001, description = "Lambda W", readout=True,readout_format='.3f'),
         c_w   =widgets.IntSlider  (value= 4182,min=0.200, max=5000,     step=1, description = "Cw"),
         rho_w =widgets.FloatSlider(value= 1000,  min=900,    max=1200, step=0.1, description = "Rho_w",readout=True,readout_format='.3f'),
         ne =widgets.FloatSlider(value=0.25,  min=0.01,    max=0.8, step=0.01, description = "ne",readout=True,readout_format='.2f'),
         x1  =widgets.IntSlider  (value=   25,    min=1,  max=500,     step=1, description = "<= Ort (x in m)"),
         t2  =widgets.IntSlider  (value=  182,    min=1,  max=365,     step=1, description = "=> Zeit (t in d)"));


interactive(children=(FloatSlider(value=8.0, description='Temp. ini (C)', max=15.0, min=4.0), FloatSlider(valu…